In [1]:
# %% [markdown]
# # Real Estate Synthetic Data Generator
#
# This notebook generates synthetic data for a real estate application, including:
# - **Buyers:** Prospective buyers with financial profiles.
# - **Houses for Sale:** Listings in Central NJ.
# - **Past Sales:** Historical sales records.
# - **Basic Transcripts:** Simulated call transcripts (rule-based).
# - **LLM Transcripts:** Simulated call transcripts (using OpenAI, if API key is configured).
#
# Data will be saved to a specified output directory.

# %%
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
from faker import Faker
import random
import uuid
import datetime
import os
import time
from openai import OpenAI # Only if generating LLM transcripts
from IPython.display import display, Markdown # For Jupyter display
from dotenv import load_dotenv
load_dotenv()

# Initialize Faker
fake = Faker('en_US')

# Configuration
NUM_BUYERS = 5000
NUM_HOUSES_FOR_SALE = 1000
NUM_PAST_SALES = 500
NUM_TRANSCRIPTS_BASIC = 10000 # For basic rule-based transcripts
# WARNING: Setting NUM_TRANSCRIPTS_LLM high will take significant time and API cost.
# Start with a small number like 100 for testing.
NUM_TRANSCRIPTS_LLM = 100 # Set desired number for LLM transcripts

HOUSE_PRICE_MIN = 500_000
HOUSE_PRICE_MAX = 4_000_000

OUTPUT_DIR = "real_estate_synthetic_data"

# Central NJ Towns (Example - can be expanded)
CENTRAL_NJ_TOWNS = [
    "Princeton", "West Windsor", "Plainsboro", "Montgomery", "Hillsborough",
    "Bridgewater", "Edison", "Woodbridge", "East Brunswick", "South Brunswick",
    "Franklin Township", "Piscataway", "New Brunswick", "Hopewell", "Lawrenceville"
]

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory '{OUTPUT_DIR}' ensured.")

# OpenAI Client Setup (Conditional)
openai_api_key = os.getenv("OPENAI_API_KEY")
client = None
llm_enabled = False
if openai_api_key:
    try:
        client = OpenAI(api_key=openai_api_key)
        llm_enabled = True
        print("OpenAI client initialized successfully.")
    except Exception as e:
        print(f"Failed to initialize OpenAI client: {e}. LLM transcript generation will be skipped.")
        llm_enabled = False
else:
    print("OPENAI_API_KEY environment variable not set. LLM transcript generation will be skipped.")

# Helper function to get output dir (used by save functions)
def get_output_dir():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    return OUTPUT_DIR

# %% [markdown]
# ## Configuration Summary
#
# - **Buyers:** {NUM_BUYERS}
# - **Houses for Sale:** {NUM_HOUSES_FOR_SALE}
# - **Past Sales:** {NUM_PAST_SALES}
# - **Basic Transcripts:** {NUM_TRANSCRIPTS_BASIC}
# - **LLM Transcripts:** {NUM_TRANSCRIPTS_LLM} (if enabled)
# - **Output Directory:** `{OUTPUT_DIR}`

# %%


Output directory 'real_estate_synthetic_data' ensured.
OpenAI client initialized successfully.


In [2]:

# Cell 2: Helper Functions

def generate_credit_score(income_bracket):
    """Generates a credit score, loosely correlated with income bracket."""
    if income_bracket == 'Very High':
        base, std = 780, 40
    elif income_bracket == 'High':
        base, std = 740, 60
    elif income_bracket == 'Medium':
        base, std = 680, 80
    else: # Low
        base, std = 620, 100

    # Add possibility of outliers
    if random.random() < 0.05: # 5% chance of a score outside the norm
        score = np.random.randint(300, 850)
    else:
        score = int(np.random.normal(base, std))

    return np.clip(score, 300, 850) # Clip to valid range

def generate_financials(income_bracket):
    """Generates correlated income, net worth, and debt based on bracket."""
    if income_bracket == 'Very High':
        income = np.random.randint(300_000, 1_500_000)
        net_worth_multiplier = random.uniform(2, 10)
        debt_ratio = random.uniform(0.1, 0.5)
    elif income_bracket == 'High':
        income = np.random.randint(150_000, 300_000)
        net_worth_multiplier = random.uniform(1.5, 7)
        debt_ratio = random.uniform(0.2, 0.6)
    elif income_bracket == 'Medium':
        income = np.random.randint(80_000, 150_000)
        net_worth_multiplier = random.uniform(0.5, 4)
        debt_ratio = random.uniform(0.3, 0.8)
    else: # Low
        income = np.random.randint(40_000, 80_000)
        net_worth_multiplier = random.uniform(0.1, 2)
        debt_ratio = random.uniform(0.4, 1.0) # Can exceed income

    net_worth = int(income * net_worth_multiplier + np.random.normal(0, income * 0.2)) # Add noise
    net_worth = max(0, net_worth) # Ensure non-negative net worth

    total_debt = int(income * debt_ratio + np.random.normal(0, income * 0.1))
    total_debt = max(0, total_debt)

    # Outlier check: high debt regardless of income
    if random.random() < 0.03:
        total_debt = int(income * random.uniform(1.0, 2.5))

    # Outlier check: Low net worth despite high income
    if income_bracket in ['High', 'Very High'] and random.random() < 0.05:
         net_worth = int(income * random.uniform(0.1, 0.5))
         net_worth = max(0, net_worth)

    return income, net_worth, total_debt

def generate_fake_address():
    """Generates a somewhat more realistic Central NJ address."""
    street = fake.street_address()
    town = random.choice(CENTRAL_NJ_TOWNS)
    # Faker might produce zips outside NJ, this is a simplification
    zipcode = fake.zipcode_in_state(state_abbr='NJ')
    return f"{street}, {town}, NJ {zipcode}"

def generate_ssn():
    """Generates a valid-looking SSN."""
    return fake.ssn()

print("Helper functions defined.")

# %%


Helper functions defined.


In [3]:

# Cell 3: Generate Buyers Dataset

def generate_buyers(num_buyers):
    buyers_data = []
    income_brackets = ['Low', 'Medium', 'High', 'Very High']
    bracket_probabilities = [0.15, 0.45, 0.30, 0.10] # Example distribution
    print(f"Generating {num_buyers} buyer records...")
    for i in range(num_buyers):
        if (i + 1) % 500 == 0:
            print(f"  Generated {i + 1}/{num_buyers} buyers...")
        buyer_id = str(uuid.uuid4())
        full_name = fake.name()
        address = generate_fake_address()
        ssn = generate_ssn()
        phone_number = fake.phone_number()
        email = fake.email()

        income_bracket = np.random.choice(income_brackets, p=bracket_probabilities)
        annual_income, net_worth, total_debt = generate_financials(income_bracket)
        credit_score = generate_credit_score(income_bracket)

        # Desired price range based on financials, with variation
        base_desired = annual_income * random.uniform(3, 7) # Simple heuristic
        desired_min = int(max(HOUSE_PRICE_MIN * 0.8, base_desired * 0.7))
        desired_max = int(min(HOUSE_PRICE_MAX * 1.2, base_desired * 1.3))
        # Ensure min < max and within bounds
        desired_min = min(desired_min, HOUSE_PRICE_MAX)
        desired_max = max(desired_min + 50000, desired_max) # Ensure some range
        desired_max = min(desired_max, HOUSE_PRICE_MAX*1.1) # Allow slight overshoot
        desired_min = max(desired_min, HOUSE_PRICE_MIN*0.9)

        # Simple Pre-Approval Model
        max_monthly_payment = (annual_income / 12) * random.uniform(0.35, 0.50)
        credit_factor = 1.0 + (credit_score - 700) / 1000
        estimated_loan = max(0, max_monthly_payment * 200 * credit_factor - total_debt * 0.5)
        pre_approved_amount = int(np.clip(estimated_loan, 0, HOUSE_PRICE_MAX * 1.5))

        # Introduce outliers in pre-approval
        if random.random() < 0.05:
            pre_approved_amount = int(pre_approved_amount * random.uniform(0.5, 2.0))
            pre_approved_amount = max(0, pre_approved_amount)

        buyers_data.append({
            "BuyerID": buyer_id,
            "FullName": full_name,
            "Address": address,
            "SSN": ssn,
            "PhoneNumber": phone_number,
            "Email": email,
            "IncomeBracket": income_bracket,
            "AnnualIncome": annual_income,
            "NetWorth": net_worth,
            "TotalDebt": total_debt,
            "CreditScore": credit_score,
            "DesiredPriceRange_Min": desired_min,
            "DesiredPriceRange_Max": desired_max,
            "PreApprovedAmount": pre_approved_amount
        })

    df = pd.DataFrame(buyers_data)
    print(f"Finished generating {len(df)} buyer records.")
    return df

def create_buyers_df():
    df_buyers = generate_buyers(NUM_BUYERS)
    # Save to CSV
    buyers_csv_path = os.path.join(get_output_dir(), "buyers.csv")
    df_buyers.to_csv(buyers_csv_path, index=False)
    print(f"Buyers data saved to: {buyers_csv_path}")
    return df_buyers, buyers_csv_path

# Execute buyer generation
df_buyers, buyers_csv_path = create_buyers_df()

# Display header
display(Markdown(f"### Buyers Dataset ({len(df_buyers)} records)"))
display(Markdown(f"Saved to: `{buyers_csv_path}`"))
display(df_buyers.head())

# %%


Generating 5000 buyer records...
  Generated 500/5000 buyers...
  Generated 1000/5000 buyers...
  Generated 1500/5000 buyers...
  Generated 2000/5000 buyers...
  Generated 2500/5000 buyers...
  Generated 3000/5000 buyers...
  Generated 3500/5000 buyers...
  Generated 4000/5000 buyers...
  Generated 4500/5000 buyers...
  Generated 5000/5000 buyers...
Finished generating 5000 buyer records.
Buyers data saved to: real_estate_synthetic_data/buyers.csv


### Buyers Dataset (5000 records)

Saved to: `real_estate_synthetic_data/buyers.csv`

,BuyerID,FullName,Address,SSN,PhoneNumber,Email,IncomeBracket,AnnualIncome,NetWorth,TotalDebt,CreditScore,DesiredPriceRange_Min,DesiredPriceRange_Max,PreApprovedAmount
0,a78eee35-7bd0-4053-89f5-75b69a2bfe6f,Jesse Gonzalez,"93049 Gardner Lake Apt. 701, Franklin Township...",284-68-3890,399.287.8216,mary73@example.org,Low,52499,75171,26145,677,450000.0,450000.0,340984
1,c59a9498-f39c-4a8c-ba96-d905a83e10b3,Martin Dunn MD,"164 Billy Trace, Plainsboro, NJ 07133",515-45-1852,001-878-693-7507x603,david93@example.com,Medium,138014,73760,75092,631,614221.0,1140697.0,862017
2,d6892e8e-a99a-4fe6-ae3c-17d26f7a0ef5,Gina Diaz,"5723 Natalie Course Suite 473, East Brunswick,...",748-75-6576,323-273-2409,cassandraroberson@example.com,Medium,92364,131177,35393,696,450000.0,603808.0,710891
3,56d4e3cd-cd3e-4249-acb8-32106756f7ba,Deanna Romero,"924 Richard Way Suite 780, Plainsboro, NJ 08337",465-32-6968,+1-768-958-8777x20401,william79@example.org,High,255854,1557898,65561,669,716259.0,1330195.0,1454933
4,4bb94344-80a2-4d85-a627-7ce5cc047aef,Kristi Smith,"2829 Jennifer Canyon Suite 033, Princeton, NJ ...",456-21-6726,001-232-752-2247x9016,rachaelkelley@example.com,Medium,122669,486509,29713,643,546548.0,1015017.0,889114


In [4]:

# Cell 4: Generate Houses for Sale Dataset

def generate_houses(num_houses):
    houses_data = []
    property_types = ['Single Family', 'Townhouse', 'Condo', 'Multi-Family']
    type_probabilities = [0.65, 0.15, 0.10, 0.10]
    print(f"Generating {num_houses} house listings...")

    for i in range(num_houses):
        if (i + 1) % 200 == 0:
             print(f"  Generated {i + 1}/{num_houses} houses...")
        house_id = str(uuid.uuid4())
        address = generate_fake_address()
        listing_price = np.random.randint(HOUSE_PRICE_MIN, HOUSE_PRICE_MAX + 1)

        # Correlate features with price (loosely)
        price_factor = (listing_price - HOUSE_PRICE_MIN) / (HOUSE_PRICE_MAX - HOUSE_PRICE_MIN)

        bedrooms = np.random.randint(2, 7) + int(price_factor * 2)
        bedrooms = max(2, bedrooms)

        bathrooms = round(np.random.uniform(1.5, 5.0) + price_factor * 2, 1)
        bathrooms = max(1.5, min(bathrooms, 6.0))
        bathrooms = round(bathrooms * 2) / 2

        base_sqft = 1000
        sqft = int(base_sqft + (bedrooms * 250) + (bathrooms * 150) + (price_factor * 3000) + np.random.normal(0, 300))
        sqft = max(800, sqft)

        lot_size = round(np.random.uniform(0.05, 3.0) + price_factor * 2.0, 2)
        lot_size = max(0.05, lot_size)

        year_built = np.random.randint(1940, datetime.datetime.now().year + 1)
        if price_factor > 0.7 and random.random() < 0.6:
            year_built = np.random.randint(1990, datetime.datetime.now().year + 1)
        elif price_factor < 0.3 and random.random() < 0.6:
            year_built = np.random.randint(1940, 1985)

        property_type = np.random.choice(property_types, p=type_probabilities)
        if property_type == 'Condo' and listing_price > 1_000_000:
             listing_price = int(listing_price * random.uniform(0.6, 0.9))
        elif property_type == 'Single Family' and listing_price < 700_000:
             listing_price = int(listing_price * random.uniform(1.0, 1.3))
        listing_price = np.clip(listing_price, HOUSE_PRICE_MIN, HOUSE_PRICE_MAX)

        houses_data.append({
            "HouseID": house_id,
            "Address": address,
            "ListingPrice": listing_price,
            "Bedrooms": bedrooms,
            "Bathrooms": bathrooms,
            "SquareFootage": sqft,
            "LotSize_Acres": lot_size,
            "YearBuilt": year_built,
            "PropertyType": property_type,
            "Status": "For Sale" # Assuming this function generates 'For Sale' listings
        })

    df = pd.DataFrame(houses_data)
    print(f"Finished generating {len(df)} house listings.")
    return df

def create_houses_df():
    df_houses = generate_houses(NUM_HOUSES_FOR_SALE)
    # Save to CSV
    houses_csv_path = os.path.join(get_output_dir(), "houses_for_sale.csv")
    df_houses.to_csv(houses_csv_path, index=False)
    print(f"Houses data saved to: {houses_csv_path}")
    return df_houses, houses_csv_path

# Execute house generation
df_houses, houses_csv_path = create_houses_df()

# Display header
display(Markdown(f"### Houses for Sale Dataset ({len(df_houses)} records)"))
display(Markdown(f"Saved to: `{houses_csv_path}`"))
display(df_houses.head())


# %%


Generating 1000 house listings...
  Generated 200/1000 houses...
  Generated 400/1000 houses...
  Generated 600/1000 houses...
  Generated 800/1000 houses...
  Generated 1000/1000 houses...
Finished generating 1000 house listings.
Houses data saved to: real_estate_synthetic_data/houses_for_sale.csv


### Houses for Sale Dataset (1000 records)

Saved to: `real_estate_synthetic_data/houses_for_sale.csv`

,HouseID,Address,ListingPrice,Bedrooms,Bathrooms,SquareFootage,LotSize_Acres,YearBuilt,PropertyType,Status
0,e5b2602e-e3c9-40a4-82f7-17b46b7990be,"21463 Munoz Greens Suite 700, Lawrenceville, N...",1494431,6,2.5,3662,2.07,1956,Single Family,For Sale
1,e3f9baf5-bc82-4332-87b4-56ef4054446c,"35198 Smith Point, Montgomery, NJ 08168",3932012,6,5.0,6469,2.90,2022,Single Family,For Sale
2,a55cb7d0-d745-4161-bce5-e776f6055a5a,"980 Brown Flat Apt. 079, Edison, NJ 08034",590543,6,5.0,3347,2.86,1966,Single Family,For Sale
3,9291b582-f780-4b3d-b693-517c9823479e,"293 Luis Branch Apt. 841, Franklin Township, N...",2069584,6,5.0,4471,2.07,2010,Single Family,For Sale
4,7e73a405-b1e6-4076-a762-ac78cb9dd196,"213 Mary Views, Franklin Township, NJ 08053",1631035,6,4.5,4436,2.04,2008,Condo,For Sale


In [5]:

# Cell 5: Generate Past Sales Dataset

def generate_past_sales(num_sales, buyers_df, houses_df_structure_unused):
    """ Generates past sales, linking to buyers and simulating house details """
    # NOTE: houses_df_structure_unused is kept for signature compatibility but not used as intended in the original Marimo code.
    # The original code generated NEW house details for each past sale via generate_houses(1).
    sales_data = []
    sale_notes_categories = [
        "Normal", "Job Relocation", "Downsizing", "Upsizing",
        "Divorce", "Estate Sale", "Job Loss", "Bankruptcy/Foreclosure"
    ]
    sale_notes_probabilities = [0.65, 0.10, 0.05, 0.05, 0.04, 0.04, 0.035, 0.035]

    print(f"Generating {num_sales} past sales records...")

    # Ensure we have enough buyers to sample from
    if len(buyers_df) == 0:
        print("Warning: Buyers DataFrame is empty. Cannot generate past sales.")
        return pd.DataFrame()

    if len(buyers_df) < num_sales:
        print("Warning: Not enough unique buyers generated for the number of sales. Buyers will be reused.")
        buyer_indices = np.random.choice(buyers_df.index, num_sales, replace=True)
    else:
        buyer_indices = np.random.choice(buyers_df.index, num_sales, replace=False) # Unique buyers per sale if possible

    available_buyers = buyers_df.loc[buyer_indices].copy()

    for i in range(num_sales):
        if (i + 1) % 100 == 0:
             print(f"  Generated {i + 1}/{num_sales} past sales...")

        sale_id = str(uuid.uuid4())

        # Generate *new* house details for this past sale using the same logic
        temp_house_df = generate_houses(1) # This now prints "Generating 1 house listings..." each time. Consider suppressing print inside loop.
        house_details = temp_house_df.iloc[0]

        # Select a buyer for this sale
        buyer_info = available_buyers.iloc[i]
        buyer_id = buyer_info["BuyerID"]

        # Adjust generated house price towards buyer's capability
        listing_price = house_details["ListingPrice"]
        target_price = (buyer_info['DesiredPriceRange_Min'] + buyer_info['DesiredPriceRange_Max']) / 2
        target_price = min(max(target_price, HOUSE_PRICE_MIN), HOUSE_PRICE_MAX) # Clamp target

        adjusted_listing_price = int(listing_price * 0.5 + target_price * 0.5 + np.random.normal(0, listing_price * 0.1))
        adjusted_listing_price = np.clip(adjusted_listing_price, HOUSE_PRICE_MIN, HOUSE_PRICE_MAX)

        # Determine Sale Price
        sale_price_ratio = random.uniform(0.93, 1.07)
        sale_price = int(adjusted_listing_price * sale_price_ratio)
        if random.random() < 0.08:
            sale_price = int(adjusted_listing_price * random.uniform(0.85, 1.15))
        sale_price = np.clip(sale_price, int(HOUSE_PRICE_MIN*0.8), int(HOUSE_PRICE_MAX*1.1))

        # Sale Date
        sale_date = fake.date_between(start_date="-5y", end_date="today")

        # Sale Notes
        sale_category = np.random.choice(sale_notes_categories, p=sale_notes_probabilities)
        sale_details = f"Standard transaction."
        # (Logic for different sale categories and potential price adjustments)
        if sale_category == "Job Relocation":
            sale_details = f"Seller relocated for a new job opportunity in {fake.city()}."
            if random.random() < 0.3: sale_price = int(sale_price * random.uniform(0.92, 0.98))
        elif sale_category == "Downsizing":
            sale_details = "Seller downsizing after retirement/children moved out."
        elif sale_category == "Upsizing":
            sale_details = "Seller buying a larger home for growing family."
            if random.random() < 0.2: sale_price = int(sale_price * random.uniform(1.01, 1.05))
        elif sale_category == "Divorce":
            sale_details = "Sale resulting from divorce proceedings."
            if random.random() < 0.4: sale_price = int(sale_price * random.uniform(0.90, 0.97))
        elif sale_category == "Estate Sale":
            sale_details = "Property sold as part of an estate settlement."
            if random.random() < 0.5: sale_price = int(sale_price * random.uniform(0.88, 1.02))
        elif sale_category == "Job Loss":
            sale_details = "Forced sale due to unexpected job loss and financial hardship."
            sale_price = int(adjusted_listing_price * random.uniform(0.85, 0.95))
        elif sale_category == "Bankruptcy/Foreclosure":
            sale_details = "Sale managed through bankruptcy court or bank foreclosure process."
            sale_price = int(adjusted_listing_price * random.uniform(0.80, 0.93))

        sale_price = max(int(HOUSE_PRICE_MIN*0.75), sale_price) # Final price floor

        sales_data.append({
            "SaleID": sale_id,
            "HouseID": house_details["HouseID"], # Link to the generated house's ID
            "BuyerID": buyer_id,
            "SellerFullName": fake.name(), # Generate fake seller
            "SellerAddress": generate_fake_address(), # Fake seller address
            "ListingPrice": adjusted_listing_price, # Use adjusted price
            "SalePrice": sale_price,
            "SaleDate": sale_date,
            "SaleCategory": sale_category,
            "SaleDetails": sale_details,
            # Buyer snapshot
            "Buyer_AnnualIncome": buyer_info["AnnualIncome"],
            "Buyer_NetWorth": buyer_info["NetWorth"],
            "Buyer_TotalDebt": buyer_info["TotalDebt"],
            "Buyer_CreditScore": buyer_info["CreditScore"],
             # House snapshot
            "House_Address": house_details["Address"],
            "House_Bedrooms": house_details["Bedrooms"],
            "House_Bathrooms": house_details["Bathrooms"],
            "House_SquareFootage": house_details["SquareFootage"],
            "House_YearBuilt": house_details["YearBuilt"],
            "House_PropertyType": house_details["PropertyType"],
        })

    df = pd.DataFrame(sales_data)
    print(f"Finished generating {len(df)} past sales records.")
    return df

def create_sales_df():
    # Pass buyers df and the structure (columns) of houses df
    # Suppress the inner print from generate_houses(1) if desired, or ignore it.
    df_sales = generate_past_sales(NUM_PAST_SALES, df_buyers, df_houses) # df_houses is passed but not used as intended
    # Save to CSV
    sales_csv_path = os.path.join(get_output_dir(), "past_sales.csv")
    if not df_sales.empty:
        df_sales.to_csv(sales_csv_path, index=False)
        print(f"Past sales data saved to: {sales_csv_path}")
    else:
        print("Past sales DataFrame is empty. No file saved.")
        sales_csv_path = None
    return df_sales, sales_csv_path

# Execute past sales generation
df_sales, sales_csv_path = create_sales_df()

# Display header
if sales_csv_path:
    display(Markdown(f"### Past Sales Dataset ({len(df_sales)} records)"))
    display(Markdown(f"Saved to: `{sales_csv_path}`"))
    display(df_sales.head())
else:
    display(Markdown("### Past Sales Dataset Generation Skipped (likely no buyers)"))


# %%


Generating 500 past sales records...
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listings.
Generating 1 house listings...
Finished generating 1 house listing

### Past Sales Dataset (500 records)

Saved to: `real_estate_synthetic_data/past_sales.csv`

,SaleID,HouseID,BuyerID,SellerFullName,SellerAddress,ListingPrice,SalePrice,SaleDate,SaleCategory,SaleDetails,Buyer_AnnualIncome,Buyer_NetWorth,Buyer_TotalDebt,Buyer_CreditScore,House_Address,House_Bedrooms,House_Bathrooms,House_SquareFootage,House_YearBuilt,House_PropertyType
0,4473e46d-92a2-48e1-8533-082c24c05645,4007df26-4a45-4e9c-a250-4cd26bd6b6b3,a351aa39-d3d1-49ff-b386-d589db3ee5c1,Linda Bradford,"3021 Johnny Hills Apt. 457, South Brunswick, N...",2062468,2028892,2022-08-09,Normal,Standard transaction.,107614,341245,27657,670,"72625 Cooper Point Apt. 762, Princeton, NJ 08459",4,6.0,5692,1960,Multi-Family
1,cb369b11-9d89-45fe-a8a4-a61fbc60b321,8479281b-d2f3-41bd-899f-7336fb6322f4,c1587235-4aa1-4535-887f-721d8f452402,Billy Salas,"110 Perez Glen, New Brunswick, NJ 07942",2442142,2211793,2023-03-09,Estate Sale,Property sold as part of an estate settlement.,115806,368683,39375,675,"706 Lawrence Ways Apt. 152, Plainsboro, NJ 07365",3,6.0,5303,2025,Single Family
2,e73fa076-05ea-4b11-8a22-d24ccc52d4e6,56cd2f8f-8345-4462-befb-031e6fb8d507,8adc0c5d-ac83-456a-a126-d604852070de,Sherri Love,"76198 Christina View Apt. 697, West Windsor, N...",1360961,1380643,2022-01-17,Normal,Standard transaction.,219766,1352840,43104,848,"259 Luis Springs Apt. 423, Plainsboro, NJ 07610",2,5.0,3291,1983,Condo
3,2dd51277-7b44-401c-a4c3-9d4a1b65728d,6a180ee1-5800-4224-9ca6-50d30375665b,7e67af1e-6391-4daf-bc6c-6aa1a6b2bc57,Mrs. Jodi Cruz,"897 Rebecca Summit Suite 068, Lawrenceville, N...",2763080,2779271,2022-10-24,Normal,Standard transaction.,526624,2963563,194612,844,"02076 Morales Extensions, Edison, NJ 08416",5,5.0,4766,2007,Single Family
4,d3cfbb03-6c6d-4c23-b69e-80de80533841,ebe3205f-9698-4a5c-9b1d-30578a954dfe,415d0181-0ecd-459f-bf06-78f49b8eb3ee,Adrian Strickland,"08141 West Ports, Montgomery, NJ 08236",2046474,2092341,2024-05-21,Upsizing,Seller buying a larger home for growing family.,221464,816258,50066,725,"748 Hensley Village, Bridgewater, NJ 08987",4,6.0,5352,1955,Townhouse


In [6]:

# Cell 6: Generate Basic Call Transcripts (Rule-Based)

def generate_basic_transcripts(num_transcripts, buyers_df):
    transcripts_data = []
    broker_names = [fake.name() for _ in range(25)] # Pool of brokers
    banker_names = [fake.name() for _ in range(25)] # Pool of bankers

    print(f"Generating {num_transcripts} basic call transcripts...")

    # Ensure we have buyers to sample from
    if len(buyers_df) == 0:
        print("Warning: Buyers DataFrame is empty. Cannot generate basic transcripts.")
        return pd.DataFrame()

    buyer_ids = buyers_df['BuyerID'].tolist()

    for i in range(num_transcripts):
        if (i + 1) % 1000 == 0:
             print(f"  Generated {i + 1}/{num_transcripts} basic transcripts...")

        transcript_id = str(uuid.uuid4())
        call_datetime = fake.date_time_between(start_date="-2y", end_date="now")
        buyer_id = random.choice(buyer_ids)

        try:
            buyer_info = buyers_df[buyers_df['BuyerID'] == buyer_id].iloc[0]
        except IndexError:
            print(f"Warning: BuyerID {buyer_id} not found in buyers_df. Skipping transcript.")
            continue # Skip if buyer_id not found

        broker_name = random.choice(broker_names)
        banker_name = random.choice(banker_names)

        # Extract PII
        buyer_name = buyer_info['FullName']
        buyer_address = buyer_info['Address']
        buyer_ssn = buyer_info['SSN']
        buyer_ssn_last4 = buyer_ssn.split('-')[-1]
        buyer_phone = buyer_info['PhoneNumber']
        buyer_income = buyer_info['AnnualIncome']
        buyer_desired_max = buyer_info['DesiredPriceRange_Max']

        # Simple transcript templates including PII
        templates = [
            f"MB: Hi {buyer_name}, this is {banker_name}. Just confirming your application details for the mortgage - is your SSN still {buyer_ssn}? \nBuyer: Yes, that's correct. \nMB: Great, and the address {buyer_address} is current?",
            f"Broker: Hello {buyer_name}, {broker_name} calling. Regarding houses around ${buyer_desired_max:,.0f}, I have a new listing you might like. \nBuyer: Oh really? Tell me more. \nBroker: It's on Maple St, let's connect later. Your number is {buyer_phone}, right?",
            f"MB: {banker_name} here for {buyer_name}. We need to verify income for the pre-approval. \nBuyer: Okay, what do you need? \nMB: Can you confirm your full SSN {buyer_ssn} and current residence at {buyer_address} for security?",
            f"Broker: {broker_name} checking in with {buyer_name}. Any thoughts on the properties we saw last week? \nBuyer: Still considering. The one near {random.choice(CENTRAL_NJ_TOWNS)} park was nice. \nBroker: Got it. Just confirming your details for updates: Name: {buyer_name}, Address: {buyer_address}, SSN: {buyer_ssn}.",
            f"MB: {buyer_name}, it's {banker_name}. The underwriter needs clarification on your debt-to-income ratio, given your stated income of ${buyer_income:,.0f}. \nBuyer: Okay, what specifically? \nMB: Let's review your file. Confirming SSN ending in {buyer_ssn_last4} and address {buyer_address}."
        ]

        transcript_text = random.choice(templates)

        transcripts_data.append({
            "TranscriptID": transcript_id,
            "CallDateTime": call_datetime,
            "BuyerID": buyer_id,
            "BrokerName": broker_name,
            "MortgageBankerName": banker_name,
            "TranscriptText": transcript_text
        })

    df = pd.DataFrame(transcripts_data)
    print(f"Finished generating {len(df)} basic transcripts.")
    return df

def create_basic_transcripts_df():
    df_basic_transcripts = generate_basic_transcripts(NUM_TRANSCRIPTS_BASIC, df_buyers)
    # Save to CSV
    basic_transcripts_csv_path = os.path.join(get_output_dir(), "basic_call_transcripts.csv")
    if not df_basic_transcripts.empty:
        df_basic_transcripts.to_csv(basic_transcripts_csv_path, index=False)
        print(f"Basic transcripts saved to: {basic_transcripts_csv_path}")
    else:
        print("Skipping save for empty basic transcripts DataFrame.")
        basic_transcripts_csv_path = None
    return df_basic_transcripts, basic_transcripts_csv_path

# Execute basic transcript generation
df_basic_transcripts, basic_transcripts_csv_path = create_basic_transcripts_df()

# Display header
if basic_transcripts_csv_path:
    display(Markdown(f"### Basic Call Transcripts Dataset ({len(df_basic_transcripts)} records)"))
    display(Markdown(f"Saved to: `{basic_transcripts_csv_path}`"))
    display(df_basic_transcripts.head())
else:
    display(Markdown("### Basic Call Transcripts Generation Skipped or Resulted in Empty Data"))


# %%


Generating 10000 basic call transcripts...
  Generated 1000/10000 basic transcripts...
  Generated 2000/10000 basic transcripts...
  Generated 3000/10000 basic transcripts...
  Generated 4000/10000 basic transcripts...
  Generated 5000/10000 basic transcripts...
  Generated 6000/10000 basic transcripts...
  Generated 7000/10000 basic transcripts...
  Generated 8000/10000 basic transcripts...
  Generated 9000/10000 basic transcripts...
  Generated 10000/10000 basic transcripts...
Finished generating 10000 basic transcripts.
Basic transcripts saved to: real_estate_synthetic_data/basic_call_transcripts.csv


### Basic Call Transcripts Dataset (10000 records)

Saved to: `real_estate_synthetic_data/basic_call_transcripts.csv`

,TranscriptID,CallDateTime,BuyerID,BrokerName,MortgageBankerName,TranscriptText
0,a10761c1-a847-464b-aa31-49cbb3788336,2024-04-18 14:51:10.284777,1a51eadb-1b15-4fea-94d2-e233f0d3ff49,Jose Fox,Samantha Larson,"MB: Hi Michelle Martin, this is Samantha Larso..."
1,6bf80df4-e335-4235-b896-30d4069c844f,2023-06-07 14:56:02.180452,ade6b200-51f1-45f4-a973-c9f2b599f2f3,Jose Fox,George Lawrence,"Broker: Hello Tommy Mason, Jose Fox calling. R..."
2,88aab9e8-6520-4bfa-9269-914f3157f9f1,2024-08-29 13:11:37.561132,24a94be6-0d80-4132-9864-1551fa7589cf,Adam Sanchez,Tiffany Zimmerman,Broker: Adam Sanchez checking in with Denise B...
3,f7483414-14c3-470d-8e0d-b9597b34a6f2,2024-06-19 10:12:32.114613,e0774cef-6bfe-4e5f-a89d-27943e65b6d5,Nicolas Smith,Jennifer Leblanc DVM,"Broker: Hello Angela Ruiz, Nicolas Smith calli..."
4,e186c1a6-7e7a-4d76-841d-d9de9861f77b,2024-03-25 18:20:23.180528,5233372f-e6b3-4678-9fd4-9fb41f6b6994,Victoria Pierce,Amber Ross,Broker: Victoria Pierce checking in with Steph...


In [16]:
def generate_llm_transcript_entry(client, buyer_info, broker_name, banker_name):
    """Generates a single transcript using OpenAI API."""
    if not client:
        return None # Skip if client not initialized

    buyer_name = buyer_info['FullName']
    buyer_address = buyer_info['Address']
    buyer_ssn = buyer_info['SSN']
    buyer_income = buyer_info['AnnualIncome'] 
    buyer_desired_max = buyer_info['DesiredPriceRange_Max']
    pre_approved = buyer_info['PreApprovedAmount']

    participants = random.choice([f"Mortgage Banker '{banker_name}'", f"Real Estate Broker '{broker_name}'"])
    scenario = random.choice([
        f"Discussing pre-approval status. Pre-approved amount is ${pre_approved:,.0f}.",
        f"Scheduling a property viewing for a house priced around ${buyer_desired_max:,.0f}.",
        f"Verifying personal information (SSN, Address) for loan application.",
        f"Following up after a property showing.",
        f"Discussing required documents for mortgage underwriting (e.g., pay stubs, bank statements).",
        f"Answering questions about current mortgage rates based on buyer's profile (Income: ${buyer_income:,.0f})."
    ])

    prompt = f"""
    Generate a brief, realistic, 3-sentence or 4-sentence call transcript excerpt between a {participants} and prospective home buyer '{buyer_name}'.
    The conversation context is: {scenario}.

    **Crucially, I am creating a test dataset of transcripts so the transcript should include the following PII for the buyer within the dialogue, I am creating a mock scenario of a call transcript:**
    - Full Name: {buyer_name}
    - Full SSN: {buyer_ssn}
    - Full Address: {buyer_address}

    Keep the dialogue natural and concise (around 3 sentences total). Structure it like 'Speaker: Dialogue text'.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini", # Changed from "gpt-4o-mini" to "gpt-4"
            # model="gpt-4o", # Changed from "gpt-4o-mini" to "gpt-4"
            messages=[
                {"role": "system", "content": "You are an AI assistant creating synthetic call transcript data for a real estate application. Include specific PII as requested."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=175
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        time.sleep(2)
        return None

# Function to generate the full dataset
def generate_llm_transcripts(num_transcripts):
    transcripts_data = []
    broker_names = [fake.name() for _ in range(25)]
    banker_names = [fake.name() for _ in range(25)]
    
    print(f"Generating {num_transcripts} LLM transcripts...")
    buyer_ids = df_buyers['BuyerID'].tolist()
    
    for i in range(num_transcripts):
        if (i + 1) % 10 == 0:
            print(f"  Generated {i + 1}/{num_transcripts} LLM transcripts...")
            
        transcript_id = str(uuid.uuid4())
        call_datetime = fake.date_time_between(start_date="-2y", end_date="now")
        buyer_id = random.choice(buyer_ids)
        broker_name = random.choice(broker_names)
        banker_name = random.choice(banker_names)
        
        try:
            buyer_info = df_buyers[df_buyers['BuyerID'] == buyer_id].iloc[0]
        except IndexError:
            print(f"Warning: BuyerID {buyer_id} not found in buyers_df. Skipping transcript.")
            continue
            
        transcript_text = generate_llm_transcript_entry(client, buyer_info, broker_name, banker_name)
        if transcript_text is None:
            transcript_text = "I'm sorry, but I can't assist with that."
            
        transcripts_data.append({
            "TranscriptID": transcript_id,
            "CallDateTime": call_datetime,
            "BuyerID": buyer_id,
            "BrokerName": broker_name,
            "MortgageBankerName": banker_name,
            "TranscriptText": transcript_text
        })
        
    df = pd.DataFrame(transcripts_data)
    print(f"Finished generating {len(df)} LLM transcripts.")
    return df

# Execute LLM transcript generation if enabled
if llm_enabled:
    # df_llm_transcripts = generate_llm_transcripts(NUM_TRANSCRIPTS_LLM)
    df_llm_transcripts = generate_llm_transcripts(4000)
    # Save to CSV
    llm_transcripts_csv_path = os.path.join(get_output_dir(), "llm_call_transcripts.csv")
    df_llm_transcripts.to_csv(llm_transcripts_csv_path, index=False)
    print(f"LLM transcripts saved to: {llm_transcripts_csv_path}")
    
    # Display header
    display(Markdown(f"### LLM Call Transcripts Dataset ({len(df_llm_transcripts)} records)"))
    display(Markdown(f"Saved to: `{llm_transcripts_csv_path}`"))
    display(df_llm_transcripts.head())
else:
    print("LLM transcript generation skipped (not enabled).")


Generating 4000 LLM transcripts...
  Generated 10/4000 LLM transcripts...
  Generated 20/4000 LLM transcripts...
  Generated 30/4000 LLM transcripts...
  Generated 40/4000 LLM transcripts...
  Generated 50/4000 LLM transcripts...
  Generated 60/4000 LLM transcripts...
  Generated 70/4000 LLM transcripts...
  Generated 80/4000 LLM transcripts...
  Generated 90/4000 LLM transcripts...
  Generated 100/4000 LLM transcripts...
  Generated 110/4000 LLM transcripts...
  Generated 120/4000 LLM transcripts...
  Generated 130/4000 LLM transcripts...
  Generated 140/4000 LLM transcripts...
  Generated 150/4000 LLM transcripts...
  Generated 160/4000 LLM transcripts...
  Generated 170/4000 LLM transcripts...
  Generated 180/4000 LLM transcripts...
  Generated 190/4000 LLM transcripts...
  Generated 200/4000 LLM transcripts...
  Generated 210/4000 LLM transcripts...
  Generated 220/4000 LLM transcripts...
  Generated 230/4000 LLM transcripts...
  Generated 240/4000 LLM transcripts...
  Generated 25

### LLM Call Transcripts Dataset (4000 records)

Saved to: `real_estate_synthetic_data/llm_call_transcripts.csv`

,TranscriptID,CallDateTime,BuyerID,BrokerName,MortgageBankerName,TranscriptText
0,438f3f08-9f7f-4302-bcc0-b1c348b90e87,2024-04-27 13:16:46.934386,396656e4-4336-48af-93a3-aa9a37a172b2,Angela Wilkinson,William Colon,"William Colon: Hi Jeffrey, this is William Col..."
1,8230d959-a506-4094-8280-668570120aa2,2024-04-06 20:25:40.447627,dc7c7afa-fde3-4428-ae67-60ce88e9dc7c,Lawrence Curtis,Ralph Ortiz,"Ralph Ortiz: Hi Lisa, before we proceed with y..."
2,12eb9585-1213-4052-9f5a-a9596a348d53,2023-12-25 14:55:26.193998,be342d36-9b1a-4923-b9c3-74676f2bd575,Anthony Davis,Robert Sandoval,"Anthony Davis: Hi Christopher, this is Anthony..."
3,81fa25af-9c20-41a0-ab17-0083d4b99164,2023-05-08 22:20:15.481882,81f264cb-4508-4677-b5d5-81bd8d7d9c70,Martha Wiggins,Elijah Hernandez,"Martha Wiggins: Hi Patricia Walker, based on y..."
4,a854f271-496f-4629-8693-e051efa52d27,2023-10-01 22:44:52.721801,2f60a81f-2a92-4662-ae00-6b7bdd59448c,Robert Watts,Amy Fisher,"Amy Fisher: Hi Ashley, to move forward with yo..."


In [15]:

# Cell 8: Summary

display(Markdown(f"""
## Data Generation Complete

The following datasets have been generated and saved in the `{OUTPUT_DIR}` directory:

1.  **Buyers:** `{os.path.basename(buyers_csv_path) if buyers_csv_path else '*Not Generated*'}` ({len(df_buyers)} records)
2.  **Houses for Sale:** `{os.path.basename(houses_csv_path) if houses_csv_path else '*Not Generated*'}` ({len(df_houses)} records)
3.  **Past Sales:** `{os.path.basename(sales_csv_path) if sales_csv_path else '*Not Generated*'}` ({len(df_sales)} records)
4.  **Basic Transcripts:** `{os.path.basename(basic_transcripts_csv_path) if basic_transcripts_csv_path else '*Not Generated*'}` ({len(df_basic_transcripts)} records)
5.  **LLM Transcripts:** {f'`{os.path.basename(llm_transcripts_csv_path)}` ({len(df_llm_transcripts)} records)' if llm_transcripts_csv_path else '*Skipped or Not Generated*'}
"""))

print("\nScript Finished.")


## Data Generation Complete

The following datasets have been generated and saved in the `real_estate_synthetic_data` directory:

1.  **Buyers:** `buyers.csv` (5000 records)
2.  **Houses for Sale:** `houses_for_sale.csv` (1000 records)
3.  **Past Sales:** `past_sales.csv` (500 records)
4.  **Basic Transcripts:** `basic_call_transcripts.csv` (10000 records)
5.  **LLM Transcripts:** `llm_call_transcripts.csv` (100 records)



Script Finished.
